In [13]:
# https://stackoverflow.com/questions/37159070/multiple-linear-regression-model-by-using-tensorflow
# https://donaldpinckney.com/books/pytorch/book/ch2-linreg/2018-03-21-multi-variable.html
# https://www.youtube.com/watch?v=Q4GNLhRtZNc
# https://atmamani.github.io/projects/ml/coursera-gd-multivariate-linear-regression/
# https://online.stat.psu.edu/stat462/sites/onlinecourses.science.psu.edu.stat462/files/05mlr/eq_matrix_notation/index.gif

# Multivariable Logistic Regression for matricies.
# target = flux1 + flux2 +... flux500 + b
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import pandas as pd
from numpy import exp

In [15]:
#retrieve and format data - into labels from the dataset
def labels(filename):
    data = pd.read_csv(filename)
    data_y = data['LABEL']
    data_y-= 1
    return data_y

In [16]:
#Logistic Layer using a sigmoid function
def logistic_layer(y):
    y = np.array(y)
    y = 1 / (1 + exp(-y)) # sigmoid function
    y = y.ravel()
    return y

In [17]:
#Calculate an accuracy metric
def accuracy(predicted_y, true_y):
    true_y = np.array(true_y).ravel()
    counter = 0
    for i in range(len(true_y)):
        p_y = predicted_y[i]
        t_y = true_y[i]
        if (p_y>.5 and t_y == 1) or (p_y < .5 and t_y == 0):
            counter+=1
    counter = (counter/ len(true_y)) * 100
    return counter

In [18]:
with open("../archive/ProcessData.npy", 'rb') as f:
        training_data_x = np.load(f)
        test_x = np.load(f)
training_data_x = training_data_x[:, 1:].transpose()
training_data_y = labels("../archive/exoTrain.csv")

#set hyperparameters & variables
learning_rate = 0.003
epochs = 50
display_step = 5
n_samples = training_data_x.shape[1]
col_num = training_data_x.shape[0]

X = tf.placeholder(tf.float32, [col_num, n_samples])
Y = tf.placeholder(tf.float32, [1, n_samples]) #resulting dimenstion of W*X matmul

In [19]:
# We want the weight vector to correspond one to one with every column
W = tf.Variable(tf.zeros([1,col_num], dtype=np.float32), name="weight")
b = tf.Variable(tf.zeros([1, ], dtype=np.float32), name="bias")

#matrix multiplication requires outer dimension of W to be equal to be equal to the inner dimension of X: 
# (1,col_num) & (col_num, num_samples) - this is why we transpose X
pred = tf.matmul(W, X) + b # y′(x,A,b)=Ax+b linear matrix equation

error = tf.reduce_sum((pred-Y)**2) / (n_samples * 2) #MSE

In [20]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

session = tf.Session()
session.run(tf.global_variables_initializer())

loss_arr = []
acc_arr = []

for t in range(epochs):
    
    _, current_loss, current_W, current_b = session.run([optimizer, error, W, b], feed_dict={
        X: training_data_x,
        Y: training_data_y
    })

    if t % display_step == 0:
        print("Epoch = %g, Loss = %g, Weights = %s, Bias = %s" % (t, current_loss, str(current_W), str(current_b)))
    
    loss_arr.append(current_loss)
    #acc_arr.append(accuracy(logistic_layer(current_W * training_data_x + current_b), training_data_y))
    acc_arr.append(accuracy(logistic_layer(np.dot(current_W, training_data_x) + current_b), training_data_y))
    
print("Optimization Finished!")

training_error = session.run(error, feed_dict={X: training_data_x, Y: training_data_y})
print("Training error=", training_error, "Weights=", session.run(W), "Bias=", session.run(b), '\n')

2021-12-14 03:15:24.599448: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 03:15:24.600815: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 03:15:24.601355: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 03:15:24.624934: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 03:15:24.625430: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Epoch = 0, Loss = 0.00363672, Weights = [[0. 0. 0. ... 0. 0. 0.]], Bias = [0.]
Epoch = 5, Loss = 0.00363594, Weights = [[-3.5349299e-09 -3.5014653e-09 -7.3507671e-09 ... -2.2187174e-09
  -1.2314756e-09 -5.0893560e-09]], Bias = [0.00010845]
Epoch = 10, Loss = 0.00363518, Weights = [[-1.3157819e-08 -1.3005013e-08 -2.4512484e-08 ... -6.0249454e-09
  -5.9331100e-09 -1.7963158e-08]], Bias = [0.00021529]
Epoch = 15, Loss = 0.00363444, Weights = [[-2.62748863e-08 -2.58878590e-08 -4.52657609e-08 ... -9.14055320e-09
  -1.28912365e-08 -3.43085915e-08]], Bias = [0.00032053]
Epoch = 20, Loss = 0.00363373, Weights = [[-4.1474863e-08 -4.0817682e-08 -6.7662597e-08 ... -1.1098098e-08
  -2.0843252e-08 -5.2214041e-08]], Bias = [0.00042422]
Epoch = 25, Loss = 0.00363303, Weights = [[-5.7960722e-08 -5.7060934e-08 -9.0931607e-08 ... -1.1821540e-08
  -2.9074384e-08 -7.0755299e-08]], Bias = [0.00052636]
Epoch = 30, Loss = 0.00363236, Weights = [[-7.52728226e-08 -7.41878026e-08 -1.14703894e-07 ... -1.13723697

In [23]:
test_x = test_x[:, 1:].transpose()
test_y = labels("../archive/exoTest.csv")

predicted_y = np.dot(current_W, training_data_x) + session.run(b)
predicted_y = logistic_layer(predicted_y)

print("Accuracy percentage: ", accuracy(predicted_y, test_y), "%")

Accuracy percentage:  0.8771929824561403 %


In [24]:
with open('tf-models/multi-lr.npy', 'wb') as f:
    np.save(f, session.run(W))
    np.save(f, session.run(b))
    np.save(f, loss_arr)
    np.save(f, acc_arr)
